# Querying From Multiple Tables

Until this lecture we have seen writing SQL queries to retreive data from a single table. In this lecture we will look at retrieving data from multiple tables (which is the real strength of a relational database). We will learn about writing subqueries as well as table joins to retreive data from multiple tables. Before we learn about subqueries let us look into the **IN operator**. 

## IN operator

The IN operator is a shorthand for **multiple OR conditions**.

For example I want to select all the employee records with the last name Adams, Johnson, King, and Edwards from the employee table (Chinook). 

There are two ways to achieve this 

```sql
select * from employees where "LastName"='Adams' or "LastName"='Johnson' or "LastName"='King' or "LastName"='Edwards'
```

or using the IN operator

```sql
select * from employees where "LastName" in ('Adams','Johnson','King','Edwards')
```

As you can see the query becomes less verbose. 

Now let us try one more example

Select all film records from the film table with rating as PG-13, G, NC-17 and R

```sql
select * from film where rating in ('PG-13', 'G', 'NC-17','R')
```

Now can you write a query to retrieve all the films from the film table with rating as PG-13, G, NC-17 and R and length greater than or equal to 60 minutes. 

Before we delve into subqueries let us look into one more thing

## Table Aliases

Similar to column aliases we can also use alias name for tables, which becomes handy when you are dealing with multiple tables. Let's see a few examples

```sql
select fc.film_id,fc.category_id from film_category fc
```

Or select payment_id,customer_id from payment table where amount greater than 10

```sql
select select p.payment_id,p.customer_id from payment p where p.amount>10
```

## Subquery

Subqueries (also known as inner queries or nested queries) are a tool for performing operations in multiple steps. In simple terms "A subquery is a SELECT statement within another statement". 

Subqueries are commonly used along with FROM and WHERE. First we will look at examples where subqueries are used along with where clause.

### Subquery with WHERE clause

Suppose you want to select all English films from film table.

If you check the film table there is no seperate column for language. But there seems to be a column called language_id. How will you identify which is the language id for English. 

You can look at the language table 

The language table has three columns

language_id, name and last_update

If you check the language table you can find that the corresponding id for language 'English' (which is 1)

So you could write a query like this

```sql
select * from film where language_id = 1
```

But what if the language table was huge and it is not easy to find out the id for English.

We can write this as a subquery

```sql
select * from film where language_id = (select  language_id from language where name = 'English')
```

You can see that there is another perfectly valid SQL query inside an SQL statement

The general syntax of a subquery when used along with a WHERE clause is

```sql
select * from table where column = (select column from table2 where ....)
```

You can also use IN clause with subqueries

For example let's select all the addresses that are with in all the cities from 'United States'. 

This involves three tables (there are multiple subqueries here).
```sql
select * from address where city_id in (select city_id from city where country_id = (select country_id from country where country = 'United States'))
```

When writing queries with subqueries understand that the innermost subquery will be executed first followed the next innermost subquery and finally the main query. In the above example, the innermost subquery *select country_id from country where country = 'United States'* will be executed first which will result in a single value 103. Then that value will be placed in the next subquery select city_id from city where country_id = 103 which will result in a set of 35 values, and then finally those values will be used to select the records from the address table. 

Let us see try one more example and move on to joins. 

Find all unique customers who had alteast one rental to their name. 

Give it a try!!

## Joins

We have learned about joins while dealing with dataframes in Pandas. The table joins in SQL are also very similar to dataframe joins and the basic concept remains the same. The power of joins when compared to subquery is that the columns that are available with the subquery when used in a WHERE clause is not available to the main query. 

### Inner Join

![innerjoin](../L5/images/innerjoin.png)

Inner join only retains the records based on the keys that are common to both the tables.

Let us look at an example. 

We want to retreive city name and corresponding country name from the city and country table.

```sql
select ci.city, co.country from city ci join country co on ci.country_id = co.country_id
```

Inner joins can also be written without explicit join statements. So this is also valid

```sql
select ci.city, co.country from city ci,country co where ci.country_id = co.country_id
```

But when you are not explicitly mentioning the keyword join you should not use **on**, rather the condition should be in a where clause

Now what if we want only cities from United States

```sql
select ci.city, co.country from city ci join country co on ci.country_id = co.country_id where co.country = 'United States'
```

or 
```sql
select ci.city, co.country from city ci,country co where ci.country_id = co.country_id and co.country = 'United States'
```

Now lets look at an example involving more than two tables. 

We want to select the customer first name, last name, address ,city and country. How can we achieve this

We can see that there are four tables

customer

address

city

country

Now how can we connect them together

customer  has address_id which could be used to connect to address table, while address has city_id and city has country_id

So with that knowledge we can write a join query

```sql
select cu.first_name,cu.last_name,a.address,ci.city, co.country from customer cu,address a,city ci,country co where cu.address_id = a.address_id and a.city_id = ci.city_id and ci.country_id = co.country_id
```

## Left Join
![leftjoin](../L5/images/leftjoin.png)

Left Join returns all records from the left table, and only matching records from the right table. 

Let us select all the films with their inventory details. The two tables we are considering here are 

films
inventory


```sql
select f.title,i.inventory_id,i.store_id from film f left join inventory i on f.film_id = i.film_id
```

Now if you want to check the movies that are not in the inventory, you can modify the query to

```sql
select f.title,i.inventory_id,i.store_id from film f left join inventory i on f.film_id = i.film_id where i.store_id is null
```

## Right Join

![rightjoin](../L5/images/rightjoin.png)

Right Join returns all records from the right table, and only the matching records from the left table.

Let us rewrite the same example as left join

```sql
select f.title,i.inventory_id,i.store_id from inventory i right join film f on f.film_id = i.film_id
```

## FULL Outer Join

![outerjoin](../L5/images/outerjoin.png)

Full Outer join returns unmatched records from both tables,as well as matched records in both tables. 

We won't be dealing with full outer join for this lecture. 

## Subquery in from

Let us see an example of using subquery along with from statement

Suppose we want to select customer details and the total amount paid by each customer. 

One way to do it is using a subquery in combination with join. We know that the customer details are in customer table and the payment details are in payment table. 

```sql
select c.customer_id,c.first_name,c.last_name,cu.total from customer c,
(select customer_id,sum(amount) as total from payment group by customer_id) cu where c.customer_id = cu.customer_id
```